 # P4 : Anticipez les besoins en consommation électrique de bâtiments

In [1]:
import os
import wget
import pandas as pd


In [2]:
write_data = True

# True : création d'un dossier Figures et Tableau
# dans lesquels seront créés les éléments qui serviront à la présentation
# et écriture des figures et tableaux dans ces dossier
#
# False : pas de création de dossier ni de figures ni de tableaux

if write_data is True:
    try:
        os.mkdir('./Figures/')
    except OSError as error:
        print(error)
    try:
        os.mkdir('./Tableaux/')
    except OSError as error:
        print(error)
else:
    print("""Visualisation uniquement dans le notebook
    pas de création de figures ni de tableaux""")


[Errno 17] File exists: './Figures/'
[Errno 17] File exists: './Tableaux/'


 Importation des données si nécessaire

In [3]:
if write_data is True:
    if os.path.exists('BEB2015.csv'):
        print("Le jeu de données est déjà présent dans le répertoire")
    else:
        wget.download("https://data.seattle.gov/api/views/h7rm-fz6m/rows.csv",
                      out='BEB2015.csv')
    if os.path.exists('BEB2016.csv'):
        print("Le jeu de données est déjà présent dans le répertoire")
    else:
        wget.download("https://data.seattle.gov/api/views/2bpz-gwpy/rows.csv",
                      out='BEB2016.csv')


Le jeu de données est déjà présent dans le répertoire
Le jeu de données est déjà présent dans le répertoire


In [4]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe
# sinon récupération avec l'url
if os.path.exists('BEB2015.csv'):
    BEB2015 = pd.read_csv('BEB2015.csv')
else:
    BEB2015 = pd.read_csv(
        "https://data.seattle.gov/api/views/h7rm-fz6m/rows.csv")


In [5]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe
# sinon récupération avec l'url
if os.path.exists('BEB2015.csv'):
    BEB2016 = pd.read_csv('BEB2016.csv')
else:
    BEB2016 = pd.read_csv(
        "https://data.seattle.gov/api/views/2bpz-gwpy/rows.csv")


In [6]:
BEB2015.sort_values('OSEBuildingID', inplace=True)
BEB2015.reset_index(inplace=True, drop=True)


 Liste des colonnes présentes uniquement dans les données de 2015

In [7]:
# colonnes uniquement dans les données de 2015
BEB2015.columns.difference(BEB2016.columns)


Index(['Comment', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensity(kgCO2e/ft2)', 'Location', 'OtherFuelUse(kBtu)'],
      dtype='object')

 Liste des colonnes présentes uniquement dans les données de 2016

In [8]:
# colonnes uniquement dans les données de 2016
BEB2016.columns.difference(BEB2015.columns)


Index(['Address', 'City', 'Comments', 'GHGEmissionsIntensity', 'Latitude',
       'Longitude', 'State', 'TotalGHGEmissions', 'ZipCode'],
      dtype='object')

 Les données de location en 2015 sont dans une seule colonne
 on va faire en sorte d'uniformiser avec les colonnes présentes en 2016

In [9]:
BEB2015['Latitude'] = BEB2015.Location.str.split(
    '(', expand=True)[1].str.split(',', expand=True)[0].astype('float64')
BEB2015['Longitude'] = BEB2015.Location.str.split(
    '(',
    expand=True)[1].str.split(',',
                              expand=True)[1].str.strip(')').astype('float64')
BEB2015['Address'] = BEB2015.Location.str.split('\n', n=1, expand=True)[0]
BEB2015['City'] = BEB2015.Location.str.split(
    '\n', n=1, expand=True)[1].str.split(',', n=1, expand=True)[0]
BEB2015['State'] = BEB2015.Location.str.split(
    '\n', n=1, expand=True)[1].str.split(
        ',', n=1, expand=True)[1].str.lstrip(' ').str.split(' ',
                                                            n=1,
                                                            expand=True)[0]
BEB2015['ZipCode'] = BEB2015.Location.str.split(
    '\n', n=1, expand=True)[1].str.split(
        ',', n=1, expand=True)[1].str.lstrip(' ').str.split(
            ' ', n=1,
            expand=True)[1].str.split('\n', expand=True)[0].astype('float64')


In [10]:
BEB2015.drop(columns='Location', inplace=True)


In [11]:
BEB2015.columns.difference(BEB2016.columns)

Index(['Comment', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensity(kgCO2e/ft2)', 'OtherFuelUse(kBtu)'],
      dtype='object')

In [12]:
BEB2016.columns.difference(BEB2015.columns)

Index(['Comments', 'GHGEmissionsIntensity', 'TotalGHGEmissions'], dtype='object')

 GHGEMissions (MetricTonsCO2e) et TotalGHGEmissions renseignent les mêmes informations
 ainsi que GHGEmissionsIntesity (kgCO2e/ft2) et GHGEmissionsIntensity

In [13]:
BEB2015.rename(columns={
    'GHGEmissions(MetricTonsCO2e)': 'TotalGHGEmissions',
    'GHGEmissionsIntensity(kgCO2e/ft2)': 'GHGEmissionsIntensity'
},
               inplace=True)


In [14]:
BEB2015.columns.difference(BEB2016.columns)

Index(['Comment', 'OtherFuelUse(kBtu)'], dtype='object')

In [15]:
BEB2016.columns.difference(BEB2015.columns)

Index(['Comments'], dtype='object')

In [16]:
BEB2016.Comments.unique()


array([nan])

 Pas de commentaire dans les données de 2016

In [17]:
BEB2016.drop(columns='Comments', inplace=True)

In [18]:
BEB2015.Comment.unique()

array([nan,
       'Under construction starting 6/2013 (old building demolished) and ending 9/2016. New Building re-opened 9/2016. Year built changed from 1948 to 2016.',
       "Part of McKinstry's campus, property operates a fabrication and production shop and is conditioned by natural gas overhead radiant unit heaters with no cooling. Learn more at www.mckinstry.com.",
       "One of two office buildings on McKinstry's campus partially remodeled in 2009 from warehouse to office space. Served by rooftop air handling units with underfloor VAVs.  Self-performed energy efficiency upgrades in 2015 include a long-term LED lighting re",
       'Part of McKinstry’s campus, this remodeled warehouse is mainly office but includes a full-size basketball court and gym with locker rooms, bistro with a full kitchen, and data center. Served by rooftop package units for heating and cooling. Upgrades inclu',
       'Construction completed in mid 2015. The building was unoccupied for most of 2015.',
 

 présence de commentaires dans les données de 2015

 Nous allons joindre nos données pour n'avoir qu'un seul fichier
 sur lequel travailler lors des test de modèles

In [19]:
BEBFull = BEB2015.merge(BEB2016, how='outer')